## FT084 - Introdução a Mineração de Dados
---
### Tarefa 02: Naïve Bayes, k-NN e Ensembles

Este código tem por objetivo a resolução da tarefa em questão, que consiste na implementação de 3 algoritmos de classificação, sendo eles árvore de decisão, naïve Bayes e k-NN.  
Instruções para o experimento:
1. Utilize **subamostragem aleatória** com 5 repetições para cada algoritmo e apresente o erro de classificação **médio** de cada um (para os conjuntos de testes);
2. Adote uma divisão de 70% dos dados para treinamento e 30% dos dados para teste;
3. Faça a amostragem *antes* de iniciar o treinamento e use os mesmos dados para todos os algoritmos (em cada repetição);
4. Para cada repetição, monte um **ensemble** com os classificadores já treinados (via voto majoritário), aplique ao conjunto de testes e apresente o desempenho médio.

---

#### 1) Importação das bibliotecas  
Serão utilizados alguns pacotes para a implementação do código. São eles:
- pandas: leitura dos arquivos
- numpy, scipy: cálculo de algumas estatísticas
- sklearn: modelo de classificação, separação dos dados entre treino e teste, transformação dos atributos categóricos para numéricos (caso necessário), matriz de confusão e avaliação do erro
- graphviz, io, IPython e pydotplus: visualização da árvore
- matplotly.pyplot e plotly: visualizações extras

In [1]:
# Importação das Bibliotecas
import pandas as pd
import numpy as np
from scipy import stats
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
import graphviz
from sklearn.tree import export_graphviz
from io import StringIO
from IPython.display import Image  
import pydotplus
import matplotlib.pyplot as plt
import plotly.express as px

#### 2) Criação do dataset
Será gerado um objeto do tipo dataframe com a base de dados bupa.data, utilizando os nomes dos atributos disposíveis em bupa.names.

In [2]:
columns_names = [
    'mcv',
    'alkphos',
    'sgpt',
    'sgot',
    'gammagt',
    'drinks',
    'selector'
]

In [3]:
# Criação do dataset
dataset = pd.read_table("bupa.data", sep = ",", header = None, names = columns_names)
dataset

,mcv,alkphos,sgpt,sgot,gammagt,drinks,selector
0,85,92,45,27,31,0.0,1
1,85,64,59,32,23,0.0,2
2,86,54,33,16,54,0.0,2
3,91,78,34,24,36,0.0,2
4,87,70,12,28,10,0.0,2
...,...,...,...,...,...,...,...
340,99,75,26,24,41,12.0,1
341,96,69,53,43,203,12.0,2
342,98,77,55,35,89,15.0,1
343,91,68,27,26,14,16.0,1


#### 3) Análise da base de dados
Analisar o tipo dos dados da base (numéricos ou categóricos), se há valores faltantes, e se é necessário realizar alguma transformação prévia.

In [4]:
# Observando o tamanho da base de dados
dataset.shape

(345, 7)

In [5]:
# Utilizando o método describe() para analisar a base de dados e validar se todos os atributos são numéricos
dataset.describe()

,mcv,alkphos,sgpt,sgot,gammagt,drinks,selector
count,345.000000,345.000000,345.000000,345.000000,345.000000,345.000000,345.000000
mean,90.159420,69.869565,30.405797,24.643478,38.284058,3.455072,1.579710
std,4.448096,18.347670,19.512309,10.064494,39.254616,3.337835,0.494322
min,65.000000,23.000000,4.000000,5.000000,5.000000,0.000000,1.000000
25%,87.000000,57.000000,19.000000,19.000000,15.000000,0.500000,1.000000
50%,90.000000,67.000000,26.000000,23.000000,25.000000,3.000000,2.000000
75%,93.000000,80.000000,34.000000,27.000000,46.000000,6.000000,2.000000
max,103.000000,138.000000,155.000000,82.000000,297.000000,20.000000,2.000000


In [6]:
# Criando um vetor para verificar se há valores nulos
null_array = [dataset.iloc[:, i].isnull().unique() for i in range(7)]
null_array

[array([False]),
 array([False]),
 array([False]),
 array([False]),
 array([False]),
 array([False]),
 array([False])]

In [7]:
# Criando um vetor para verificar se há valores faltantes
na_array = [dataset.iloc[:, i].isna().unique() for i in range(7)]
na_array

[array([False]),
 array([False]),
 array([False]),
 array([False]),
 array([False]),
 array([False]),
 array([False])]

#### 4) Transformação da base de dados
Classificar a coluna "drinks" de acordo com as informações do arquivo bupa.names, além de analisar a coluna "selector", avaliando se pode ser usada como divisor dos dados.
- "drinks" 3 5 pode ser considerado um tipo de seleção na base de dados (de acordo com esse artigo: McDermott & Forsyth 2016, Diagnosing a disorder in a classification benchmark, Pattern Recognition Letters, Volume 73.)
- "selector" divide a base de dados em uma determina proporção; será avaliada essa proporção, e se ela é satisfatória

In [8]:
# Classificando a coluna "drinks"
dataset.loc[dataset['drinks'] <= 3, 'drinks'] = 0
dataset.loc[dataset['drinks'] > 3, 'drinks'] = 1
dataset

,mcv,alkphos,sgpt,sgot,gammagt,drinks,selector
0,85,92,45,27,31,0.0,1
1,85,64,59,32,23,0.0,2
2,86,54,33,16,54,0.0,2
3,91,78,34,24,36,0.0,2
4,87,70,12,28,10,0.0,2
...,...,...,...,...,...,...,...
340,99,75,26,24,41,1.0,1
341,96,69,53,43,203,1.0,2
342,98,77,55,35,89,1.0,1
343,91,68,27,26,14,1.0,1


In [9]:
# Analisando a coluna "selector"
dataset.groupby(['selector']).size()

selector
1    145
2    200
dtype: int64

In [10]:
# Porcentagem de cada divisão
label_1 = 145/345
label_2 = 200/345
display(label_1)
display(label_2)

0.42028985507246375

0.5797101449275363

In [11]:
# Como a proporção é diferente de 30% e 70%, a coluna "selector" será descartada do dataset.
dataset.drop(columns = 'selector', inplace = True)
columns_names.pop(-1)
dataset

,mcv,alkphos,sgpt,sgot,gammagt,drinks
0,85,92,45,27,31,0.0
1,85,64,59,32,23,0.0
2,86,54,33,16,54,0.0
3,91,78,34,24,36,0.0
4,87,70,12,28,10,0.0
...,...,...,...,...,...,...
340,99,75,26,24,41,1.0
341,96,69,53,43,203,1.0
342,98,77,55,35,89,1.0
343,91,68,27,26,14,1.0


#### 5) Divisão entre Classe e Atributos
Definir dentro do dataset qual é a variável que será classificada, e quais são as variáveis preditoras.

In [12]:
# Definição das Classes
classes = dataset['drinks']

# Definição dos atributos
attributes_names = [
    'mcv',
    'alkphos',
    'sgpt',
    'sgot',
    'gammagt'
]

attributes = dataset[attributes_names]

#### 6) Separação dos Dados
Aqui, os dados serão separados em treinamento e teste. O parâmetro *test_size* será definido primeiramente como 0.2 (20%) de acordo com o enunciado da tarefa.

In [13]:
# Divisão da base de dados entre treinamento e teste. Random_state = 0 para sempre obter a mesma divisão da base quando o código for executado.
x_train, x_test, y_train, y_test = train_test_split(attributes, classes, test_size = 0.3, random_state = 0)

#### 7) Árvore de Decisão
Criaçao de uma função que irá encapsular o processo de criação do modelo de árvore de decisão, retornando a classe prevista para cada instância e o erro de classificação do algoritmo.

In [14]:
# Serão utilizados como parâmetros da função os valores de x e y para teste e treinamento
def decision_tree(x_train, x_test, y_train, y_test):
    # Construção do modelo
    tree = DecisionTreeClassifier()
    tree.fit(x_train, y_train)
    
    # Obtenção das previsões
    prediction = tree.predict(x_test)
    
    # Matriz de confusão
    confusion = confusion_matrix(y_test, prediction)
    
    # Taxa acerto
    accuracy = accuracy_score(y_test, prediction)

    # Taxa erro
    error = 1 - accuracy
    
    return prediction, error

#### 8) Naïve Bayes
Criaçao de uma função que irá encapsular o processo de criação do modelo de Naïve Bayes, retornando a classe prevista para cada instância e o erro de classificação do algoritmo.

In [15]:
# Serão utilizados como parâmetros da função os valores de x e y para teste e treinamento
def naive_bayes(x_train, x_test, y_train, y_test):
    # Construção do modelo
    naive_bayes = GaussianNB()
    naive_bayes.fit(x_train, y_train)
    
    # Obtenção das previsões
    prediction = naive_bayes.predict(x_test)
    
    # Matriz de confusão
    confusion = confusion_matrix(y_test, prediction)
    
    # Taxa acerto
    accuracy = accuracy_score(y_test, prediction)

    # Taxa erro
    error = 1 - accuracy
    
    return prediction, error

#### 9) k-NN
Criaçao de uma função que irá encapsular o processo de criação do modelo de k-NN, retornando a classe prevista para cada instância e o erro de classificação do algoritmo.

In [16]:
# Serão utilizados como parâmetros da função os valores de x e y para teste e treinamento
def k_nn(x_train, x_test, y_train, y_test):
    # Construção do modelo
    knn = KNeighborsClassifier(n_neighbors = 3)
    knn.fit(x_train, y_train)
    
    # Obtenção das previsões
    prediction = knn.predict(x_test)
    
    # Matriz de confusão
    confusion = confusion_matrix(y_test, prediction)
    
    # Taxa acerto
    accuracy = accuracy_score(y_test, prediction)

    # Taxa erro
    error = 1 - accuracy
    
    return prediction, error

#### 10) Ensemble
Criação de uma função que irá encapsular o processo de montagem de um Ensemble com os classificadores treinados (via voto majoritário).

In [17]:
def ensemble(decision_tree, naive_bayes, k_nn, test_array):
    # Coletando as previsões para cada um dos algoritmos e agrupando em tuplas
    list_of_predictions = [(decision_tree[i], naive_bayes[i], k_nn[i]) for i in range(len(test_array))]
    
    # Encontrando a moda de cada tupla
    prediction_list = [stats.mode(i)[0][0] for i in list_of_predictions]
    
    # Matriz de confusão
    confusion = confusion_matrix(y_test, prediction_list)
    
    # Taxa acerto
    accuracy = accuracy_score(y_test, prediction_list)

    # Taxa erro
    error = 1 - accuracy
    
    return error

#### 11) Realização do Experimento
O experimento consiste em 5 iterações para cada um dos algoritmos (incluindo o Ensemble), e avaliar o erro de classificação médio para cada um deles.

In [18]:
# Inicializando 4 vetores para armazenamento dos erros
decision_tree_error = []
naive_bayes_error = []
k_nn_error = []
ensemble_error = []

# Criando um vetor de nomenclatura de linhas
rows_names_error = [
    'Árvore de Decisão',
    'Naïve Bayes',
    'k-NN',
    'Ensemble'
]

# Criando um vetor de nomenclatura das coluna
column_names_error = ['Erro de Classificação Médio']

In [19]:
# Realização das iterações
for i in range(5):
    # Árvore de Decisão
    experiment_decision_tree = decision_tree(x_train, x_test, y_train, y_test)
    decision_tree_error.append(experiment_decision_tree[1])
    
    # Naïve Bayes
    experiment_naive_bayes = naive_bayes(x_train, x_test, y_train, y_test)
    naive_bayes_error.append(experiment_naive_bayes[1])
    
    # k-NN
    experiment_k_nn = k_nn(x_train, x_test, y_train, y_test)
    k_nn_error.append(experiment_k_nn[1])
    
    # Ensemble
    experiment_ensemble_error = ensemble(experiment_decision_tree[0], experiment_naive_bayes[0], experiment_k_nn[0], y_test)
    ensemble_error.append(experiment_ensemble_error)

In [20]:
# Construção do vetor de respostas
instances = [np.mean(decision_tree_error), np.mean(naive_bayes_error), np.mean(k_nn_error), np.mean(ensemble_error)]

# Impressão das respostas
results = pd.DataFrame(instances, index = rows_names_error, columns = column_names_error)
results

,Erro de Classificação Médio
Árvore de Decisão,0.450000
Naïve Bayes,0.326923
k-NN,0.346154
Ensemble,0.330769
